In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('H&M_dataset.csv')

In [5]:
df.head(3)

,Unnamed: 0,productId,productName,brandName,url,price,stockState,comingSoon,colorName,isOnline,colors,colorShades,newArrival,mainCatCode,details,materials
0,0,834333001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.08343330...,49.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes with open lacing at front. Canvas ...,COMPOSITION\nSole:Thermoplastic rubber 100%\nU...
1,1,1258600003,Pile-Lined Slippers,H&M,https://www2.hm.com/en_us/productpage.12586000...,24.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_slippers,Slippers in napped fabric. Decorative seam at ...,COMPOSITION\nLining:Polyester 100%\nUpper:Poly...
2,2,1036812001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.10368120...,56.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes in soft faux suede with open lacin...,COMPOSITION\nLining and insole:Cotton 100%\nUp...


In [7]:
df.shape

(9677, 16)

In [9]:
df.isnull().sum() #checking null values

Unnamed: 0        0
productId         0
productName       0
brandName         0
url               0
price             0
stockState        0
comingSoon        0
colorName         0
isOnline          0
colors            0
colorShades    3166
newArrival        0
mainCatCode       0
details           1
materials       159
dtype: int64

In [11]:
df['Unnamed: 0'].nunique() 

9677

In [13]:
df['productId'].nunique()

7671

In [15]:
df['productName'].nunique()

3246

In [17]:
df['brandName'].unique()

array(['H&M'], dtype=object)

In [19]:
df['url'].nunique()

7671

In [21]:
df['price'].nunique()

75

In [23]:
df['stockState'].nunique()

1

In [25]:
df['comingSoon'].unique()

array([False])

In [27]:
df['isOnline'].nunique()

1

In [29]:
df['colors'].nunique()

955

In [31]:
df['colorShades'].nunique()

73

In [33]:
df['newArrival'].unique()

array([False,  True])

In [35]:
df['mainCatCode'].unique()

array(['men_shoes_dressed', 'men_shoes_slippers', 'men_shoes_sneakers',
       'men_shoes_boots', 'men_shoes_loafers', 'men_shoes',
       'men_shoes_sneakers_hightop', 'ladies_premium_selection_shoes',
       'men_shoes_sandals_espandrillos', 'ladies_shoes_ankle_boots',
       'ladies_shoes_ballerinas_flats', 'ladies_nightwear_slippers',
       'ladies_shoes_sneakers', 'ladies_shoes_heels',
       'ladies_shoes_heels_pumps', 'ladies_shoes_pumps_highheels',
       'Ladies_shoes_loafers', 'ladies_shoes_sandals_espandrillos',
       'ladies_shoes_mules', 'ladies_trousers_culottes',
       'ladies_shoes_sandals', 'men_hoodiessweatshirts_sweatshirts',
       'men_cardigansjumpers_jumpers',
       'men_trousers_trousers_regular_all',
       'men_hoodiessweatshirts_hoodies', 'men_jacketscoats_jackets',
       'men_shirts_longsleeved', 'men_jeans_loose',
       'men_jacketscoats_coats', 'men_trousers_cargo',
       'men_trousers_casual', 'men_trousers_dressed',
       'men_trousers_chinos_sli

In [37]:
df['materials'].nunique()

4511

In [39]:
df['details'].nunique()

8239

In [41]:
df.duplicated().sum() #checking duplicate

0

# Data Cleaning

In [44]:
df.drop(columns=['colorShades'],inplace=True)

In [46]:
df = df.rename(columns={'Unnamed: 0': 'index'})

In [48]:
df.dropna(inplace=True,ignore_index=True)

# Modeling

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
df1 = vect.fit_transform(df['colorName'])
df1=df1.toarray()

In [53]:
df2 = vect.fit_transform(df['colors'])
df2=df2.toarray()

In [55]:
df3 = vect.fit_transform(df['mainCatCode'])
df3=df3.toarray()

In [57]:
df4 = vect.fit_transform(df['details'])
df4=df4.toarray()

In [58]:
df5 = vect.fit_transform(df['materials'])
df5=df5.toarray()

In [61]:
df_mod = np.concatenate([df1,df2,df3,df4,df5,],axis=1)

- **cosine similerity**

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(df_mod)
print(similarity)

[[1.         0.51394166 0.95777187 ... 0.06391613 0.18300812 0.07348886]
 [0.51394166 1.         0.51837323 ... 0.0646482  0.16947978 0.07666309]
 [0.95777187 0.51837323 1.         ... 0.06921335 0.18888145 0.09342122]
 ...
 [0.06391613 0.0646482  0.06921335 ... 1.         0.18348958 0.21866108]
 [0.18300812 0.16947978 0.18888145 ... 0.18348958 1.         0.14155766]
 [0.07348886 0.07666309 0.09342122 ... 0.21866108 0.14155766 1.        ]]


In [65]:
similarity.shape

(9517, 9517)

- **Creating a list**

In [69]:
list_of_item = df['productName'].tolist()

In [71]:
Item_search = input("Search the Product:")

Search the Product: Chelsea Boots


In [73]:
import difflib
find_close_match = difflib.get_close_matches(Item_search, list_of_item)
print(find_close_match)

['Chelsea Boots', 'Chelsea Boots', 'Chelsea Boots']


In [75]:
close_match = find_close_match[0]
print(close_match)

Chelsea Boots


In [77]:
index_of_Item = df[df['productName'] == close_match]['index'].tolist()[0]
index_of_Item

6

In [79]:
similarity_score = list(enumerate(similarity[index_of_Item]))
print(similarity_score)

[(0, 0.6682317007837835), (1, 0.5346695952026566), (2, 0.6699369457382259), (3, 0.1261081003875613), (4, 0.19918256849925753), (5, 0.6002656614432568), (6, 1.0000000000000016), (7, 0.94096663126357), (8, 0.6539522146992088), (9, 0.8563567048842674), (10, 0.20402273411949548), (11, 0.2148498865390506), (12, 0.5877803147411196), (13, 0.2709677411363061), (14, 0.9126622049527175), (15, 0.5394843394871343), (16, 0.6694737451395901), (17, 0.6247036064905782), (18, 0.19192084890380304), (19, 0.20855766993309374), (20, 0.9523865397498743), (21, 0.6727216854031908), (22, 0.0918116018540017), (23, 0.22793973620934835), (24, 0.8212732551132822), (25, 0.6206116171045554), (26, 0.5678548663558513), (27, 0.6052394011173843), (28, 0.6800733888630803), (29, 0.5170766492085815), (30, 0.66746752115215), (31, 0.26551266317125843), (32, 0.4636628867423418), (33, 0.26257788809872845), (34, 0.20707110652122163), (35, 0.1346711999007352), (36, 0.22502707365883523), (37, 0.05467589977226564), (38, 0.13463697

In [81]:
a=[]
for i in similarity_score:
    if (i[1] >= 0.75):
        a.append(i[0])

In [83]:
a

[6, 7, 9, 14, 20, 24, 43, 47, 70, 73, 76, 101, 105]

In [85]:
df0=pd.DataFrame({})
for item in a:
    df0[item] = df.loc[item][['productName','price','url']]

In [87]:
df0.T

,productName,price,url
6,Chelsea Boots,49.99,https://www2.hm.com/en_us/productpage.12386850...
7,Chelsea Boots,59.99,https://www2.hm.com/en_us/productpage.10726720...
9,Chelsea Boots,59.99,https://www2.hm.com/en_us/productpage.12501160...
14,Boots,74.99,https://www2.hm.com/en_us/productpage.12500120...
20,Chelsea Boots,59.99,https://www2.hm.com/en_us/productpage.12500060...
24,Lace-Up Boots,74.99,https://www2.hm.com/en_us/productpage.12500090...
43,Lace-Up Boots,74.99,https://www2.hm.com/en_us/productpage.12501170...
47,Warm-lined Slip-on Boots,29.99,https://www2.hm.com/en_us/productpage.10194860...
70,Brushed Suede Boots,119.0,https://www2.hm.com/en_us/productpage.12595970...
73,Brushed Suede Boots,119.0,https://www2.hm.com/en_us/productpage.12595970...
